# Cyberbullying detection
## Preprocessing
<hr>

In [8]:
#Import libraries
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import re
from collections import defaultdict
from nltk.stem import WordNetLemmatizer 

from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords
from cleantext import clean

from collections import  Counter
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.tokenize import word_tokenize
import pyLDAvis.gensim

%matplotlib inline

<hr>
read the dataset
<hr>

In [2]:
df_cbd = pd.read_csv('../datasets/twitter_parsed_dataset_post_cleaning.csv')

In [3]:
df_cbd = df_cbd.drop('Unnamed: 0',axis=1)

In [4]:
df_cbd.head(20)

,index,id,text,annotation,oh_label
0,5.749487e+17,5.749487e+17,halalflaws biebervalue greenlinerzjm read cont...,none,0.0
1,5.719179e+17,5.719179e+17,shreyabafna3 idiots claim people tried stop be...,none,0.0
2,3.902558e+17,3.902558e+17,mooseoftorment call sexist go auto place id ra...,sexism,1.0
3,5.682089e+17,5.682089e+17,g0ssipsquirrelx wrong isis follows example moh...,racism,1.0
4,5.755963e+17,5.755963e+17,mkr,none,0.0
5,5.611848e+17,5.611848e+17,trobinsonnewera saudi preacher raped tortured ...,none,0.0
6,4.365631e+17,4.365631e+17,millhouse66 maureenjs nooo sexist women bad dr...,sexism,1.0
7,5.759623e+17,5.759623e+17,going make pancakesdont hve strawberries hve b...,none,0.0
8,5.725815e+17,5.725815e+17,ahtweet freebsdgirl dare feelings fantastic wa...,none,0.0
9,5.004564e+17,5.004564e+17,newmanzaa theres something wrong girl wins way...,sexism,1.0


## Vectorize data

Vectorizing is the process of encoding text as integers to create feature vectors so that machine learning algorithms can understand language. 

TF-IDF will be the selected method for vectorize. TF-IDF computes the relative frequency with which a word appears in a document compared to its frequency across all documents. It’s more useful than term frequency for identifying key words in each document (high frequency in that document, low frequency in other documents).

In [5]:
# delete nulls
df_cbd[df_cbd['text'].isnull()]

,index,id,text,annotation,oh_label
394,5.729121e+17,5.729121e+17,NaN,none,0.0
1293,5.732801e+17,5.732801e+17,NaN,none,0.0
4442,5.740652e+17,5.740652e+17,NaN,none,0.0
4607,5.714734e+17,5.714734e+17,NaN,none,0.0
4787,4.888138e+17,4.888138e+17,NaN,none,0.0
6143,4.257325e+17,4.257325e+17,NaN,none,0.0
6470,5.764388e+17,5.764388e+17,NaN,none,0.0
6511,5.667005e+17,5.667005e+17,NaN,none,0.0
7551,5.739474e+17,5.739474e+17,NaN,none,0.0
8467,5.761984e+17,5.761984e+17,NaN,none,0.0


In [6]:
df_cbd['text']=df_cbd['text'].fillna('')

In [7]:
#create the list of words of every single tweet
lstWords = [row.split(' ') for row in df_cbd['text']]
lstWords

[['halalflaws',
  'biebervalue',
  'greenlinerzjm',
  'read',
  'contextno',
  'change',
  'meaning',
  'history',
  'islamic',
  'slavery'],
 ['shreyabafna3',
  'idiots',
  'claim',
  'people',
  'tried',
  'stop',
  'becoming',
  'terrorist',
  'made',
  'terrorist',
  'islamically',
  'brain',
  'dead'],
 ['mooseoftorment',
  'call',
  'sexist',
  'go',
  'auto',
  'place',
  'id',
  'rather',
  'talk',
  'guy'],
 ['g0ssipsquirrelx',
  'wrong',
  'isis',
  'follows',
  'example',
  'mohammed',
  'quran',
  'exactly'],
 ['mkr'],
 ['trobinsonnewera',
  'saudi',
  'preacher',
  'raped',
  'tortured',
  'five',
  'yearold',
  'daughter',
  'death',
  'released'],
 ['millhouse66', 'maureenjs', 'nooo', 'sexist', 'women', 'bad', 'drivers'],
 ['going',
  'make',
  'pancakesdont',
  'hve',
  'strawberries',
  'hve',
  'bananas',
  'mkr'],
 ['ahtweet',
  'freebsdgirl',
  'dare',
  'feelings',
  'fantastic',
  'way',
  'dehumanize',
  'someone'],
 ['newmanzaa',
  'theres',
  'something',
  'wr

In [12]:
tfidf_vect = TfidfVectorizer(analyzer='word')
X_tfidf = tfidf_vect.fit_transform(df_cbd['text'])
print(X_tfidf.shape)
print(tfidf_vect.get_feature_names_out())

(16848, 23275)
['000' '00simmerforlife' '01151900' ... 'zynga' 'zython86' 'zzzz']
